<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2021/blob/main/materialy-wyklady/bi_2021_05_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

W artykule 

Chen, Y., Li, P., & Wu, C. (2020). Doubly Robust Inference With Nonprobability Survey Samples. Journal of the American Statistical Association, 115(532), 2011–2021. https://doi.org/10.1080/01621459.2019.1677241


przedstawiono dwa podejścia do estymacji (w przypadku badania na podstawie próby):

1. dwukrokowe (kolejność nie ma znaczenia):
  + estymacja propensity scores celem uzyskania wektora parametrów $\boldsymbol{\lambda}$
  + estymacja modelu $E(y_i|\boldsymbol{x}_i)$ w celu uzyskania wektora parametrów $\boldsymbol{\beta}$
  + wyznaczenie estymatora DR

\begin{equation}
\hat{\theta}_{\mathrm{DR}}=
\overbrace{
\frac{1}{\hat{N}^A} 
\sum_{i=1}^{n^A} 
\frac{
y_{i}-m\left(\hat{\boldsymbol{\beta}};\boldsymbol{x}_{i}\right)}
{\rho\left(\hat{\boldsymbol{\lambda}};\boldsymbol{x}_{i}\right)}}^{\text{Średnia ważona reszt z modelu}} 
+
\overbrace{
\frac{1}{\hat{N}^B} 
\sum_{i=1}^{n^B} 
m\left(\hat{\boldsymbol{\beta}}; \boldsymbol{x}_{i}\right)}^{\text{Średnia z predykcji dla całej populacji}},
\end{equation}


2. jednokrokowe, w którym jednocześnie rozwiązujemy następujący układ równań

\begin{equation}
\begin{array}{c}
\frac{1}{N} \sum_{i=1}^{N} R_{i}\left\{\frac{1}{\rho\left(\boldsymbol{\lambda};\boldsymbol{x}_{i}\right)}-1\right\}\left\{y_{i}-m\left(\boldsymbol{\beta};\boldsymbol{x}_{i}\right)\right\} \boldsymbol{x}_{i}=\mathbf{0} \\ 
\frac{1}{N} \sum_{i=1}^{N} \frac{R_{i}}{\rho\left(\boldsymbol{\lambda}; \boldsymbol{x}_{i}\right)} \dot{m}\left(\boldsymbol{\beta}; \boldsymbol{x}_{i}\right)-\frac{1}{N} \sum_{i \in \mathcal{S}_{\mathrm{B}}} d_{i}^{\mathrm{B}} \dot{m}\left(\boldsymbol{\beta}; \boldsymbol{x}_{i}\right)=\mathbf{0}\end{array},
\end{equation}

gdzie $\dot{m}\left(\boldsymbol{\beta}, \boldsymbol{x}\right) = \partial m(\boldsymbol{\beta}, \boldsymbol{x}) /\partial \boldsymbol{\beta}$, czyli w przypadku modelu liniowego: $\dot{m}\left(\boldsymbol{\beta}, \boldsymbol{x}\right)=\boldsymbol{x}$. 

Przy czym podejście to zakłada znajomość całej populacji, a estymator wariancji wyznaczony jest wyłącznie dla modelu regresji logistycznej.
